<a href="https://colab.research.google.com/github/SeidaAhmed/An-application-of-Machine-Learning-to-COVID-19-data/blob/main/CIND850_Assignment_three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [29]:
%cd drive/My Drive/

[Errno 2] No such file or directory: 'drive/My Drive/'
/content/drive/My Drive


In [38]:
import pandas as pd
import numpy as np
from numpy import hstack
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Embedding
from keras import optimizers
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from scipy.ndimage.interpolation import shift
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

<ipython-input-38-c9b2b6db9d92>:15: DeprecationWarning: Please import `shift` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.interpolation import shift


In [39]:
df = pd.read_csv("hw3_text_data.csv")
df.head()

,consumer_complaint_narrative,product
0,XXXX has claimed I owe them {$27.00} for XXXX ...,Debt collection
1,Due to inconsistencies in the amount owed that...,Consumer Loan
2,In XX/XX/XXXX my wages that I earned at my job...,Mortgage
3,I have an open and current mortgage with Chase...,Mortgage
4,XXXX was submitted XX/XX/XXXX. At the time I s...,Mortgage


In [40]:
# drop any missing values
df = df.dropna(subset=['consumer_complaint_narrative', 'product'])

In [37]:
max_words = 10000
# Initialize tokenizer
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['consumer_complaint_narrative'])

# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(df['consumer_complaint_narrative'])

# Pad sequences to ensure uniform length
X = pad_sequences(sequences)

# Convert the labels (product names) to integers
product_labels = df['product'].astype('category').cat.codes
y = to_categorical(product_labels)

# Split features and labels
texts = df['consumer_complaint_narrative']
labels = df['product'].astype('category').cat.codes  # Encode product labels

tfidf = TfidfVectorizer(max_features=10000)
X = tfidf.fit_transform(texts)

# Split data: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# Train Random Forest
rf = RandomForestClassifier(max_depth=5, random_state=42)
rf.fit(X_train, y_train)

# Predict on test set
y_pred = rf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.5778


In [41]:
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
y = to_categorical(y)

# Tokenization and padding
max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=max_len)

# Train-test split (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=32, input_length=max_len))
model.add(GRU(16))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting (optional)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate on test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test Accuracy: {accuracy:.4f}")

# Best epoch based on validation accuracy
best_epoch = np.argmax(history.history['val_accuracy']) + 1
print(f"🏆 Best Epoch: {best_epoch}")


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 490ms/step - accuracy: 0.2863 - loss: 1.6031 - val_accuracy: 0.4352 - val_loss: 1.5797
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 267ms/step - accuracy: 0.4065 - loss: 1.5734 - val_accuracy: 0.4167 - val_loss: 1.5553
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 272ms/step - accuracy: 0.3882 - loss: 1.5473 - val_accuracy: 0.4074 - val_loss: 1.5323
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - accuracy: 0.4218 - loss: 1.5162 - val_accuracy: 0.4167 - val_loss: 1.5094
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step - accuracy: 0.4028 - loss: 1.4906 - val_accuracy: 0.4074 - val_loss: 1.4837
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - accuracy: 0.3934 - loss: 1.4661 - val_accuracy: 0.4167 - val_loss: 1.4575
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - accuracy: 0.4032 - loss: 1.4271 - val_accuracy: 0.4167 - val_loss: 1.4331
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - accuracy: 0.3848 - loss: 1.3990 - val_accuracy: 0.4167 - val_loss: 1.4